In [123]:
# En este código se resumirá el proceso llevado a cabo hasta lo que fue posible desarrollar a la fecha

#Se importaron las librerías a utilizar
import pandas as pd
import numpy as np
from datetime import datetime

#Se importó la información. Los títulos no se importaron, pues no funcionaba muy bien. Se decidió hacerlo de esta manera
#para luego importar los títulos de las columnas manualmente

mov=np.loadtxt('YPD_LuisBuitrago/BASE_MOVIMIENTOS.txt',skiprows=1,dtype=str)
print (mov[0])
print(type(mov))
len(mov)

['00003' '01JUL2017:00:00:00' '3340' '0,00' '$' '876.047,06' '$' '0,00'
 '0' '0' '0' '0' '0' '0' '0' '876047,06' '3340']
<class 'numpy.ndarray'>


31440

In [124]:
#Se generó una variable de nombres para luego cambiar el data type a dataframe e incluir los nombres de las columnas
#Se definió una variable 'x' y 'y', para luego ser combinadas en una sola con las columnas de saldo de crédito

nombres=['ID','FECHA_INFORMACION','SALDO_AHORROS','SALDO_FONDOS','x','SALDO_CREDITO1','y','SALDO_CREDITO2','SALDO_TARJETA','MONTO_COMPRAS1','MONTO_CAJERO1','MONTO_COMPRAS2','MONTO_CAJERO2','MONTO_ABONOS_NOMINA','INDICADOR_MORA','SALDO_ACTIVO','SALDO_PASIVO']
mov_df=pd.DataFrame(mov,columns=nombres)
print(mov_df.head())


      ID   FECHA_INFORMACION SALDO_AHORROS SALDO_FONDOS  x SALDO_CREDITO1  y  \
0  00003  01JUL2017:00:00:00          3340         0,00  $     876.047,06  $   
1  00171  01MAY2017:00:00:00   1.07013e+06         0,00  $   7.828.500,12  $   
2  00313  01OCT2017:00:00:00             0         0,00  $           0,00  $   
3  00644  01MAY2017:00:00:00       2204.88         0,00  $   8.863.156,75  $   
4  01445  01DEC2016:00:00:00        692926         0,00  $           0,00  $   

  SALDO_CREDITO2 SALDO_TARJETA MONTO_COMPRAS1 MONTO_CAJERO1 MONTO_COMPRAS2  \
0           0,00             0              0             0              0   
1           0,00             0              0             0              0   
2           0,00             0              0             0              0   
3           0,00   2.15317e+06         359023             0           8000   
4           0,00             0              0             0              0   

  MONTO_CAJERO2 MONTO_ABONOS_NOMINA INDICADOR_MORA

In [125]:
#Casi todos los datos son numéricos, por lo que no hay tanta necesidad de hacer una limpieza como en el caso anterior.
#En primer lugar se debe determinar lo que se considera como cliente_target.
#A continuación el código para cumplir con la primera condición, es decir, que reciba un valor de nómina superior a 1000000

abonos_nomina=mov_df['MONTO_ABONOS_NOMINA'].astype(float)
cond_1= abonos_nomina>=1000000
target_1 =abonos_nomina.loc[cond_1]
print(target_1.head())
print(len(target_1))
no_target_1=abonos_nomina.loc[cond_1==False]
print(no_target_1.head())
len(no_target_1)

9     1186420.0
10    1038930.0
17    3407330.0
19    1197110.0
20    1469220.0
Name: MONTO_ABONOS_NOMINA, dtype: float64
10550
0         0.0
1    897220.0
2         0.0
3    748000.0
4    606720.0
Name: MONTO_ABONOS_NOMINA, dtype: float64


20890

In [127]:
#Para la segunda condición, es necesario realizar un loop, que recorra las columnas de saldos y calcule el
#promedio de los tres meses anteriores.

#Se intentó un método similar para convertir la Fecha_Informacion en type Data, pero no se logró solucionar el
#error que no deja avanzar. 

#Acá sería segunda condición para considerar un target

fecha_edit=mov_df['FECHA_INFORMACION'].map(lambda x:str(x)[0:9])
dia_fecha_edit=fecha_edit.map(lambda x:str(x)[0:2])
mes_fecha_edit=fecha_edit.map(lambda x:str(x)[2:5])
año_fecha_edit=fecha_edit.map(lambda x:str(x)[5:9])
print(mes_fecha_edit.head())
print(año_fecha_edit.head())
months=('ENE','FEB','MAR','ABR','MAY','JUN','JUL','AGO','SEP','OCT','NOV','DEC')
months2=(1,2,3,4,5,6,7,8,9,10,11,12)
months3=('01','02','03','04','05','06','07','08','09','10','11','12')
mes_fecha_edit=mes_fecha_edit.replace(months,months3)
print(mes_fecha_edit2.head())

fecha=mes_fecha_edit.astype(str).str.cat(dia_fecha_edit.astype(str),sep='-')
fecha=fecha.str.cat(año_fecha_edit.astype(str),sep='-')

print(fecha.head())



0    JUL
1    MAY
2    OCT
3    MAY
4    DEC
Name: FECHA_INFORMACION, dtype: object
0    2017
1    2017
2    2017
3    2017
4    2016
Name: FECHA_INFORMACION, dtype: object
0    07
1    05
2    10
3    05
4    12
Name: FECHA_INFORMACION, dtype: object
0    07-01-2017
1    05-01-2017
2    10-01-2017
3    05-01-2017
4    12-01-2016
Name: FECHA_INFORMACION, dtype: object


In [128]:
fecha_edit = pd.to_datetime(fecha, format='%m-%d-%Y')


ValueError: time data 'APR-01-2017' does not match format '%m-%d-%Y' (match)

In [129]:
#Este es el código que se utilizaría para unir las columnas de saldos con las creadas 'x' y 'y' para cargar los documentos
df2=df['x'].map(str)+' ' +df['SALDO_CREDITO1']
print(df2.head())

0      $ 876.047,06
1    $ 7.828.500,12
2            $ 0,00
3    $ 8.863.156,75
4            $ 0,00
dtype: object
